In [1]:
from src import API,Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

INFO - Logger configured


In [3]:

#API call
callApi=API()
start="2023-07-12 11:03:00.000000"
end="2023-07-12 12:04:00.000000"
raw_data=callApi.getRawDataForCartoWear('C1598FF23480',pd.to_datetime(start),pd.to_datetime(end))
print(raw_data)
#Preprocessing
sampling_time=2
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_averaged_Filter(window))\
        .process()
        
 #ValueError: min() arg is an empty sequen
# dans __segmenting
# quand il y a eu delay > 100 sampling_time

INFO - data Cleaning
INFO - cleaned, sorting
INFO - sorted, filtering


                   timestamp     macModule  rssi
0    2023-07-12 10:03:59.282  A8032A311F96   -78
1    2023-07-12 10:03:58.062  A8032A311F6A   -75
2    2023-07-12 10:03:55.511  A8032A311F6A   -77
3    2023-07-12 10:03:55.503  A8032A311DAA   -80
4    2023-07-12 10:03:54.152  A8032A311F96   -79
...                      ...           ...   ...
1657 2023-07-12 09:04:11.788  A8032A311F6A   -80
1658 2023-07-12 09:04:09.242  A8032A311F6A   -78
1659 2023-07-12 09:03:30.425  A8032A311F6A   -80
1660 2023-07-12 09:03:17.253  A8032A311F6A   -79
1661 2023-07-12 09:03:12.070  A8032A311F6A   -80

[1662 rows x 3 columns]


ValueError: min() arg is an empty sequence

In [ ]:
fig=rssi_viewer(rssi_df)


In [ ]:
def get_strongest_rssi(df,timestamp,timedelta,n):
    """
    Return the n strongest rssi values at a given timestamp
    """
    start=timestamp
    end=timestamp+pd.Timedelta(seconds=timedelta)
    return df.query('@start<=timestamp<@end').sort_values('rssi',ascending=False)['macModule'].head(n)

In [ ]:
#ici on est obligé de passer de l'heure locale à l'heure UTC pour se raccorder à la bdd
import logging
logger = logging.getLogger('cartobat')

min_time=pd.to_datetime(start)-pd.Timedelta(hours=2)+pd.Timedelta(seconds=sampling_time)#permits to not have the issue of the starting data is incomplete
max_time=pd.to_datetime(end)-pd.Timedelta(hours=2)

time_range=pd.date_range(min_time,max_time,freq=pd.Timedelta(seconds=sampling_time))
number_of_mac_module=3


static_mac_module_list=get_strongest_rssi(rssi_df,min_time,sampling_time,number_of_mac_module).to_list()
print(static_mac_module_list)



[]


In [ ]:
for i in time_range:
    current_strongest=get_strongest_rssi(rssi_df,i,sampling_time,number_of_mac_module).to_list()
    j=0
    while len(static_mac_module_list)<number_of_mac_module and j<len(current_strongest):
        mac_module=current_strongest[j]
        if mac_module not in static_mac_module_list:
            static_mac_module_list.append(mac_module)
        j+=1

    different_count=0
    for j in current_strongest:
        if j not in static_mac_module_list:
            different_count+=1


 
    if different_count>=2:
        print(str(i)+"movement detected")
        #reset the static_mac_module_list
        static_mac_module_list=current_strongest
    elif different_count>=1:
        print(str(i)+"potential movement detected")

